In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = pd.read_csv('atp_tennis.csv')

In [3]:
data = data.drop(['Tournament', 'Date', 'Series'], axis=1)

In [4]:
dummy_cols = pd.get_dummies(data[['Court', 'Surface', 'Round']])

In [5]:
df = pd.concat([data, dummy_cols], axis=1)

In [6]:
df = df.drop(['Surface', 'Court', 'Round'], axis=1)

In [7]:
scores_df = df['score']

In [8]:
scores_df = scores_df.reset_index(drop=True)

In [9]:
scores_df = scores_df.to_frame(name='score')

In [10]:
#def split_scores(row):
#    scores = row['score'].split()
#    num_rounds = len(scores)
#    if num_rounds == 2:
#        player1_scores, player2_scores = scores[0].split('-'), scores[1].split('-')
#        if len(player1_scores) < 2:
#            player1_scores = [0, player1_scores[0]]
#        if len(player2_scores) < 2:
#            player2_scores = [0, player2_scores[0]]
 #   elif num_rounds == 3:
  #      player1_scores, player2_scores, _ = [score.split('-') for score in scores]
   #     if len(player1_scores) < 2:
#        player1_scores = [0, player1_scores[0], player1_scores[1]]
#     3   if len(player2_scores) < 2:
#            player2_scores = [0, player2_scores[0], player2_scores[1]]
#    else:
#        return row
    
#    new_cols = ['player{}_round{}'.format(player_num, round_num) for player_num in range(1, 3) for round_num in range(1, num_rounds + 1)]
#    return pd.concat([pd.Series(player1_scores + player2_scores), row.drop('score')]).rename(lambda x: new_cols[x - 1] if x < num_rounds * 2 else x)


In [11]:
# create example DataFrame

# apply function to each row and concatenate the resulting DataFrame with the original
#scores_df = scores_df.apply(split_scores, axis=1)

# fill missing values with empty string
#scores_df = scores_df[['player1_round1', 'player1_round2', 'player1_round3', 'player2_round1', 'player2_round2', 'player2_round3']].fillna('')


In [12]:
scores_df[['player1_round1', 'player2_round1', 'player1_round2', 'player2_round2', 'player1_round3', 'player2_round3']] = scores_df['score'].str.split('-', expand=True).fillna('')

In [13]:
#scores_df[['player2_round1', 'player1_roundtwo']] = scores_df['player2_round1'].str.split(n=1, expand=True)

In [14]:

def split_columns_on_whitespace(df):
    """
    Splits every column of a Pandas DataFrame on whitespace.
    """
    for col in df.columns:
        new_cols = df[col].str.split(expand=True)
        new_cols.columns = [f"{col}_{i+1}" for i in range(len(new_cols.columns))]
        df = pd.concat([df, new_cols], axis=1)
    df = df.drop(columns=df.columns[:len(df.columns)//2])
    return df


In [15]:
new_score_df = split_columns_on_whitespace(scores_df)

In [16]:
new_score_df = new_score_df.drop('score_5', axis=1)

In [17]:
new_score_df = new_score_df.fillna(0)

In [18]:
list(new_score_df.columns)

['player1_round1_1',
 'player2_round1_1',
 'player2_round1_2',
 'player1_round2_1',
 'player1_round2_2',
 'player2_round2_1',
 'player2_round2_2',
 'player1_round3_1',
 'player1_round3_2',
 'player2_round3_1']

In [19]:
new_score_df['player1_round1'] = new_score_df['player1_round1_1']

new_score_df['player2_round1'] = new_score_df['player2_round1_1']
new_score_df['player1_round2'] = new_score_df['player2_round1_2']
new_score_df['player2_round2'] = new_score_df['player1_round2_1']
new_score_df['player1_round3'] = new_score_df['player1_round2_2']                               
new_score_df['player2_round3'] = new_score_df['player2_round2_1']
new_score_df['player1_round4'] = new_score_df['player2_round2_2']
new_score_df['player2_round4'] = new_score_df['player1_round3_1']
new_score_df['player1_round5'] = new_score_df['player1_round3_2']
new_score_df['player2_round5'] = new_score_df['player2_round3_1']

In [20]:
new_score_df = new_score_df.drop(['player1_round1_1', 'player2_round1_1','player1_round2_1', 'player2_round1_2', 'player1_round2_2','player2_round2_1','player2_round2_2','player1_round3_1','player1_round3_2','player2_round3_1'], axis=1)

In [21]:
df = pd.concat([df, new_score_df], axis=1)

In [22]:
df = df.drop('score', axis=1)

In [23]:
df['Player_1'] = df['Player_1'].str.split(' ').str[0]
df['Player_2'] = df['Player_2'].str.split(' ').str[0]
df['Winner'] = df['Winner'].str.split(' ').str[0]

In [24]:
df.to_csv('complete_tennis.csv')